In [12]:
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from typing import Optional, Type, Callable
import requests
import json
from urllib.parse import quote

# WebサーバのAPIエンドポイント
url = "https://cir.nii.ac.jp/opensearch/all?count=5&sortorder=0&format=json&q="

# エンドポイントにPOSTリクエストを送信
def _run(url, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
    query2 = url + quote(query)  #.replace("'", '"').encode('utf-8')
    print("くえり", query2)
    # query_dict = json.loads(query)
    response = requests.get(query2)  # .post(url) # +  "get_topic", query_dict)
    print(response)
    topic_content = response.text
    print(topic_content)
    return topic_content

# 非同期実行関数の定義は必須
# async def _run(url, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
#     raise NotImplementedError("ListTopicTool does not support async")

result = _run(url, "大規模言語モデル")

import json
from pathlib import Path
from pprint import pprint

data = json.loads(result)  
for x in data:
    id = data['items']
title_list = []
for titles in id:
    # print(titles['title'])
    title_list.append(titles['title'])
print(title_list)

くえり https://cir.nii.ac.jp/opensearch/all?count=5&sortorder=0&format=json&q=%E5%A4%A7%E8%A6%8F%E6%A8%A1%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB
<Response [200]>
{"@context":{"@vocab":"http://purl.org/rss/1.0/","rdf":"http://www.w3.org/1999/02/22-rdf-syntax-ns#","rdfs":"http://www.w3.org/2000/01/rdf-schema#","dc":"http://purl.org/dc/elements/1.1/","prism":"http://prismstandard.org/namespaces/basic/2.0/","ndl":"http://ndl.go.jp/dcndl/terms","opensearch":"http://a9.com/-/spec/opensearch/1.1/","cir":"https://cir.nii.ac.jp/schema/1.0/","@language":"ja"},"@id":"https://cir.nii.ac.jp/opensearch/all?count=5&sortorder=0&format=json&q=%E5%A4%A7%E8%A6%8F%E6%A8%A1%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB","@type":"channel","title":"CiNii Research all - 5 0 json 大規模言語モデル","description":"CiNii Research all - 5 0 json 大規模言語モデル","link":{"@id":"https://cir.nii.ac.jp/opensearch/all?count=5&sortorder=0&format=json&q=%E5%A4%A7%E8%A6%8F%E6%A8%A1%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB"},"dc:da

In [11]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.llms import LlamaCpp
from langchain import hub
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import load_prompt

llm = LlamaCpp(
    model_path="llama.cpp/models/rinna-youri-7b-chat-q4_K_M.gguf",
    # model_path=r"llama.cpp/cpp\models\llama-2-7b-chat.Q4_K_M.gguf",
    input={
        "max_tokens": 32,
        "stop": ["System:", "User:", "Assistant:", "\n"],
    },
    verbose=True,
    temperature=1,
    n_ctx=2048
)

from langchain.docstore.document import Document
# 分割した長文を書く文章ごとにDocumentオブジェクト化
docs = [Document(page_content=t) for t in title_list]

map_prompt_template_path = 'map_prompt.json'
combine_prompt_template_path = 'combine_prompt.json'

chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    return_intermediate_steps=True,
    verbose=True,
    map_prompt=load_prompt(map_prompt_template_path), # load_promptはpromtの設定をファイルで読み込んでいる。json, py, yamlファイルで読み込める
    combine_prompt=load_prompt(combine_prompt_template_path)
)

output = chain(
    inputs=docs,
    return_only_outputs=True,
)["output_text"]

print(output)




c:\Users\onowa\miniconda3\envs\vir_env\lib\site-packages\langchain\utils\utils.py:159: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  warnings.warn(
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
あなたの仕事は与えられた文章から重要なエッセンスを抽出することです。文章が与えられた場合、文章に関する内容についてのエッセンスを箇条書きにして、日本語で書き出してください。

文章:PFRの家庭向けロボ、大規模言語モデル経由で操作 生成コードはLuaを採用、将来はマルチモーダル型へ
Prompt after formatting:
あなたの仕事は与えられた文章から重要なエッセンスを抽出することです。文章が与えられた場合、文章に関する内容についてのエッセンスを箇条書きにして、日本語で書き出してください。

文章:LINEヤフーが人体CG向け生成AI技術 役割を指定して2人の相互動作を出力
Prompt after formatting:
あなたの仕事は与えられた文章から重要なエッセンスを抽出することです。文章が与えられた場合、文章に関する内容についてのエッセンスを箇条書きにして、日本語で書き出してください。

文章:〔第１０１回〕　ＣＥＡＴＥＣ2023：大規模言語モデルＰＬａＭｏ、　ＭＮ−Ｃｏｒｅ、3Ｄ空間スキャン
Prompt after formatting:
あなたの仕事は与えられた文章から重要なエッセンスを抽出することです。文章が与えられた場合、文章に関する内容についてのエッセンスを箇条書きにして、日本語で書き出してください。

文章:ロボの大規模言語モデル応用が次のフェーズへ 動作計画の拘束条件をLLMで生成、学習データ不要
Prompt after formatting:
あなたの仕事は与えられた文章から重要なエッセンスを抽出することです。文章が与えられた場合、文章に関する内容についてのエッセンスを箇条書きにして、日本語で書き出してください。

文章:類似文書検索と大規模言語モデルを用いた事故記録の解析


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
あなたの仕事は箇条書きのリストを整理することです。箇条書きのリストが与えられた場合、内容が重複する箇所はひとつの文章にまとめた箇条書きのリストを書き出してください。

箇条書きのリスト:と発展

可能



 株式会社ZMPは、運行管理システム「Mission Control（MC2）」における機能向上の一環として、大規模言語モデル（LLM）を応用することで生成された動作プランが運行システムに準拠するかどうかを判定する仕組み「Rapid Planning Verification」（RPV）機能を開発しました。




Llama.generate: prefix-match hit



> Finished chain.

> Finished chain.
LLMは、最大10兆件の詳細なデータに基づいて、さまざまな形式やレベルで情報を生成および解釈することができる機械学習アプローチであり、その技術は自然言語処理と人工知能（AI）に関連しています。

LLMを活用した「RPV」の開発によって生成される動作プランにおける、運行管理システム「MC2」への準拠性や変化がなく安定的な運営を確保できます。


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import AnalyzeDocumentChain

# from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain
import os

# llm = OpenAI(temperature=0)

qa_chain = load_qa_chain(llm, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
result = qa_document_chain.run(input_document="".join(title_list), question="何についての記事ですか？")
print(result)


Llama.generate: prefix-match hit
 Science/Tech